In [9]:
import sys
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

sys.path.append('../src')

sectors = pd.read_csv('../data/reference/sector_exposure.csv')
tariff_master = pd.read_csv('../data/reference/trade_war_master.csv')

sectors.head()

,country_code,sector,us_export_value_usd,tariff_rate,exposure_level
0,CHN,Electronics,180000000000,25%,CRITICAL
1,IND,Pharmaceuticals,8500000000,26%,HIGH
2,CAN,Lumber,7000000000,25%,CRITICAL
3,MEX,Automotive,100000000000,25%,CRITICAL
4,DEU,Automobiles,25000000000,15%,HIGH


In [11]:
exposure_map = {
    'CRITICAL': 4,
    'HIGH': 3,
    'MEDIUM': 2,
    'LOW': 1
}

sectors['exposure_numeric'] = sectors['exposure_level'].map(exposure_map)

pivot = sectors.pivot_table(
    index='country_code',
    columns='sector',
    values='exposure_numeric',
    aggfunc='max'
).fillna(0)

possible_names = ['country_name', 'Country', 'country', 'Economy']
actual_col = next((name for name in possible_names if name in tariff_master.columns), None)

if actual_col:
    # Build the dictionary using the column we actually found
    code_to_name = dict(zip(tariff_master['country_code'], tariff_master[actual_col]))
    # Update the pivot index with real names
    pivot.index = [code_to_name.get(c, c) for c in pivot.index]
    print(f"✅ Successfully mapped using column: '{actual_col}'")
else:
    # Fallback: if no name column exists, just use the codes
    print(f"⚠️ Warning: No name column found. Columns are: {tariff_master.columns.tolist()}")

pivot.head()

⚠️ Warning: No name column found. Columns are: ['country_code', 'tariff_current_feb2026', 'tariff_peak', 'deal_status', 'special_notes']


sector,Automobiles,Automotive,Electronics,Lumber,Pharmaceuticals
country_code,,,,,
CAN,0.0,0.0,0.0,4.0,0.0
CHN,0.0,0.0,4.0,0.0,0.0
DEU,3.0,0.0,0.0,0.0,0.0
IND,0.0,0.0,0.0,0.0,3.0
MEX,0.0,4.0,0.0,0.0,0.0


In [12]:
fig1 = go.Figure(data=go.Heatmap(
    z=pivot.values,           # Our numeric risk levels (1-4)
    x=pivot.columns,          # The Industry Sectors
    y=pivot.index,            # The Country Names
    colorscale=[
        [0, '#f0f0f0'],      # 0 = No data (Light Gray)
        [0.25, '#2ca02c'],   # 1 = LOW (Green)
        [0.5, '#ffdd57'],    # 2 = MEDIUM (Yellow)
        [0.75, '#ff7f0e'],   # 3 = HIGH (Orange)
        [1.0, '#d62728']     # 4 = CRITICAL (Red)
    ],
    text=[[['', 'LOW', 'MEDIUM', 'HIGH', 'CRITICAL'][int(v)] 
           for v in row] for row in pivot.values],
    texttemplate='%{text}',
    textfont={'size': 9},
    hovertemplate=(
        'Country: %{y}<br>'
        'Sector: %{x}<br>'
        'Risk Level: %{text}<extra></extra>'
    )
))
fig1.update_layout(
    title='<b>Sector-Country Risk Matrix (Feb 2026)</b><br>'
          '<sup>Which industries in which countries face the highest tariff exposure?</sup>',
    xaxis_title='Sector',
    yaxis_title='Country',
    height=500,
    xaxis={'tickangle': -30}
)

fig1.show()
fig1.write_html('../reports/viz5_sector_risk_heatmap.html')

In [18]:
high_risk = sectors[sectors['exposure_level'].isin(['CRITICAL', 'HIGH'])].copy()
high_risk['trade_value_bn'] = high_risk['us_export_value_usd'] / 1e9

possible_names = ['country_name', 'Country', 'country', 'Economy']
actual_name_col = next((name for name in possible_names if name in tariff_master.columns), None)

if actual_name_col:
    high_risk = high_risk.merge(tariff_master[['country_code', actual_name_col]], on='country_code')
    high_risk['label'] = high_risk[actual_name_col] + ' — ' + high_risk['sector']
else:
    high_risk['label'] = high_risk['country_code'] + ' — ' + high_risk['sector']

if 'notes' not in high_risk.columns:
    high_risk['notes'] = ""

high_risk = high_risk.sort_values('trade_value_bn', ascending=True)
color_map = {'CRITICAL': '#d62728', 'HIGH': '#ff7f0e'}

fig2 = go.Figure(go.Bar(
    x=high_risk['trade_value_bn'],
    y=high_risk['label'],
    orientation='h',
    marker_color=[color_map[e] for e in high_risk['exposure_level']],
    text=[f"${v:.0f}B — {e}" 
          for v, e in zip(high_risk['trade_value_bn'], 
                         high_risk['exposure_level'])],
    textposition='outside',
    hovertext=high_risk['notes'],
    hovertemplate='<b>%{y}</b><br>Trade Value: $%{x:.0f}B<br>%{hovertext}<extra></extra>'
))

fig2.update_layout(
    title='<b>Highest Risk Sector-Country Pairs by Trade Value (Feb 2026)</b><br>'
          '<sup>Size = annual US trade value at risk from current tariff rates</sup>',
    xaxis_title='Annual Trade Value at Risk ($USD Billions)',
    yaxis_title='',
    height=550,
    plot_bgcolor='white',
    margin=dict(l=250),
    xaxis=dict(range=[0, 230])
)

fig2.show()
fig2.write_html('../reports/viz6_sector_exposure_ranking.html')

print("✅ Sector analysis charts saved")

✅ Sector analysis charts saved
